In [ ]:
sketch_path = '/home/kyle/256x256/sketch/tx_000000001010/airplane/n02691156_10151-3.png'
features_file = 'features'

In [ ]:
import caffe
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
model_root = '../models/triplet_googlenet/Triplet_googlenet_'
pretrained_model = model_root + 'final.caffemodel' 
sketch_proto = model_root + 'sketchdeploy.prototxt'

In [ ]:
caffe.set_mode_gpu()
sketch_net = caffe.Net(sketch_proto, pretrained_model, caffe.TEST)

In [ ]:
output_layer_sketch = 'pool5/7x7_s1_s'

In [ ]:
# seems a little strange to have a transform on the sketch images?
transformer = caffe.io.Transformer({'data': np.shape(sketch_net.blobs['data'].data)})
transformer.set_mean('data', np.array([104, 117, 123]))
transformer.set_transpose('data',(2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

In [ ]:
features = np.load(features_file + '.npy')
filenames = open(features_file + '.txt').read().splitlines()
len(features), len(filenames)

In [ ]:
%time nbrs = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='cosine').fit(features)

In [ ]:
img = caffe.io.load_image(sketch_path)
print(img.min(), img.mean(), img.max())

In [ ]:
sketch = [transformer.preprocess('data', img)]
sketch = np.asarray(sketch)
results = sketch_net.forward(data=sketch)[output_layer_sketch]
results = results.reshape(-1)

In [ ]:
%time knns = nbrs.kneighbors(results.reshape(1, -1))[1].reshape(-1)
show_image(sketch_path)
for knn in knns:
    show_image(filenames[knn])

In [ ]:
from io import BytesIO
import numpy as np
import PIL.Image
import IPython.display
import shutil

def show_array(a, fmt='png', filename=None):
    a = np.uint8(np.clip(a, 0, 255))
    image_data = BytesIO()
    PIL.Image.fromarray(a).save(image_data, fmt)
    if filename is None:
        IPython.display.display(IPython.display.Image(data=image_data.getvalue()))
    else:
        with open(filename, 'w') as f:
            image_data.seek(0)
            shutil.copyfileobj(image_data, f)
            
from scipy.misc import imread

def show_image(path):
    show_array(imread(path))

In [ ]:
show_image('/home/kyle/256x256/photo/tx_000100000000/airplane/n02691156_2289.jpg')